In [ ]:
import sys, os
# Need to add parent folder to SYS PATH in order to import packages.
sys.path.insert(0, os.path.abspath('..'))

from inwards_tasks.solvency_txt_generator import SolvencyII, correct_run_off
import pandas as pd
import numpy as np

In [ ]:
def find_file(file: str, folderpath: str) -> str:
    """Return file that matches the pattern passed to file argument.

    Parameters
    ----------
    file : str
        Pattern to be searched for each file inside folderpath
    folderpath : str
        Folder path that contains the files to be searched.

    Returns
    -------
    str
        File name.

    Raises
    ------
    ValueError
        Raise error if there're 2 files with the same pattern passed to file
        argument.
    """
    filtered_file_list = [f for f in os.listdir(folderpath) if file in f]
    if len(filtered_file_list) > 1:
        err = f'The filter "{file}" returned more than 1 file: {filtered_file_list}.'
        raise ValueError(err)
    else:
        # If no error, then return single file.
        return filtered_file_list[0]
    

# Step 1 - Correct SYMY extract by removing run-off
- This step corrects the run-off that is being applyed to the exposure uploaded to SYMY.
- The Excel file passed to the function below contains only the exposures that need to be corrected to remove run-off.
- The function will then generate a file that ends with `SYMY run-off corrected.txt`. This file will be used to generate the TXT files for ITS.

In [ ]:
correct_run_off(
    symy_filepath=find_file(file='SYMY Main', folderpath=os.getcwd()),
    run_off_correction_filepath='Final LAPSED REVERSAL data - correct format-20220331.xlsx',
    to_csv=True,
    return_df=False
)

# Step 2 - Generate text files for SII run
- For the Sinosure Retro, the correction of exposures will be in the Excel file called `YYYYQX corrections.xlsx`
- The correction will be applied when function `load_data_correction()` is ran.
- The files will be generated when running `generate_txt_files()`

In [ ]:
s = SolvencyII(report_date='31/03/2022')

In [ ]:
symy_filepath = find_file(file='SYMY run-off corrected', folderpath=os.getcwd())
s.import_symy_data(symy_filepath)

register_filepath = '202204041700_2022Q1_IRE_TREATYREG_02.txt'
s.import_register_data(register_filepath)

sharepoint_filepath = '20220401__Sharepoint data_hardcoded.xlsx'
s.import_sharepoint_data(sharepoint_filepath)

symy_expiry_filepath = '20220401-SYMY expiry-date.txt'
s.import_expiry_dates_data(symy_expiry_filepath)

In [ ]:
s.load_data_correction(data_correction_filepath='2022Q1 corrections.xlsx')

In [ ]:
s.generate_txt_files(file_version='04')